In [32]:
import os
import time
from pydantic import BaseModel, Field
from typing import List, Literal, Optional
from atlassian import Jira
from langchain.agents import tool
from langchain_core.tools import StructuredTool,BaseTool
from langchain.memory import ConversationBufferMemory
from langchain.agents import AgentExecutor
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.tools.render import format_tool_to_openai_function
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.prompts import MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory.chat_message_histories import DynamoDBChatMessageHistory

## Jira API

In [33]:
from atlassian import Jira

In [34]:
JIRA_INSTANCE_URL = "https://micaelleosouza.atlassian.net/"
USER_NAME ='micaelle.oliveira10@hotmail.com'
PASSWORD ='ATATT3xFfGF05BbX6A-oX4NTTUhQ5Zo0ZUhk5YfNBdJibEDdSFsOEP9uKLZ3r9LReJLtjqsaLOeeJ46sRLDMAfAPFrE0YwLYReeznj0WdUEYfFtvZA5O6FNfoWibqQomzZQECbEQKuVZZ9qCfaLVJ3NH6i0jr29TeQ7ntA-Tx411wNz6WNfIj4o=02F8FBA7'

In [35]:
def config_jira():
    jira = Jira(
    url=JIRA_INSTANCE_URL,
    username=USER_NAME,
    password= PASSWORD,
    cloud=True)
    return jira 

jira = Jira(
    url=JIRA_INSTANCE_URL,
    username=USER_NAME,
    password= PASSWORD,
    cloud=True)

In [36]:
# Get all projects
# Returns all projects which are visible for the currently logged in user.
#jira.projects(included_archived=None, expand=None)

# Get project
#jira.project(key, expand=None)

# Get issue by key
#jira.issue(key)

# Update issue field
#fields = {'summary': 'New summary'}
#jira.update_issue_field(key, fields, notify_users=True)

# Create issue
#jira.issue_create(fields) #{{"summary": "test issue", "description": "test description", "issuetype": {{"name": "Task"}}, "priority": {{"name": "Low"}}}}

# Issues within an Epic
#jira.epic_issues(epic_key)

# Returns all epics from the board, for the given board Id.
# This only includes epics that the user has permission to view.
# Note, if the user does not have permission to view the board, no epics will be returned at all.
#jira.get_epics(board_id, done=False, start=0, limit=50, )


In [37]:
fields = {
       "project":
       {
          "key": "GENAIT"
       },
       "summary": "Teste de criação de isssue por função",
       "description": "estou testando a criação do isso do jira por meio da biblioteca do python",
       "issuetype": {
          "name": "Story"
       }
   }


# Create issue
jira.issue_create(fields)

Creating issue "Teste de criação de isssue por função"


{'id': '10169',
 'key': 'GENAIT-153',
 'self': 'https://micaelleosouza.atlassian.net/rest/api/2/issue/10169'}

## AgentJira

In [38]:
from pydantic import BaseModel
from typing import List, Literal, Optional

In [39]:
class IssueJira(BaseModel):
    project: str = Field(description="nome do projeto onde será criado o issue")
    title: str = Field(description="Nome do issue, em frase resumida com até 4 palavras")
    description: str = Field(description="descrição do issue.")
    issuetype: Literal['Task', 'Story', 'Epic'] =  Field(description="Tipo do issue a ser criado")
    #priority: Literal['Highest', 'High', 'Medium','Low','Lowest'] = Field(description="An issue's priority indicates its relative importance. ")

def generate_tools_for_user(session_id: str) -> List[BaseTool]:
    """Generate a set of tools that have a user id associated with them."""

    @tool(args_schema=IssueJira)
    def criar_issue_Jira(**dict_info:IssueJira) -> dict:
        """Chame essa função para criar issues no Jira"""
        fields = {
            "project":
            {
                "key": dict_info["project"]
            },
            "summary": dict_info["title"],
            "description": dict_info["description"],
            "issuetype": {
                "name": dict_info["issuetype"]
            }
            }
        
        try:    # Create issue
            jira = config_jira()
            result = jira.issue_create(fields)
            print(result)
        except Exception as e:
            print(e)
            if type(e).__name__ in ['MissingSchema','NameError']:
                return "Aconteceu um erro ao enviar o issue ao Jira. O usuário não configurou as credenciais da sua conta do Jira"
            else:
                return f"Aconteceu o erro {type(e).__name__} ao enviar o issue ao Jira"
        
        return result
    
    return criar_issue_Jira


In [40]:
id ='2'
criar_issue_Jira = generate_tools_for_user(id)

In [41]:
prompt = ChatPromptTemplate.from_messages([
    ("system", """Você é um especialista em documentação funcional e em Jira.
    Você escreve todo tipo de documentação funcional com base na orientação do usuário.
    Você ajuda ao usuário, em uma jornada, para criar doumentações funcionais desde do nível macro ao micro.
    Quando a documentação estiver a nível de storys, você pode enviálas ao jira, se o usuário solicitar.
    Caso seja solicitado essas informações devem ser enviadas ao Jira.
    Você não pode inventar o nome do projeto do Jira.
    Caso o usuário não informe o nome do projeto, você deve avisá-lo que ele deve adicionar o nome do projeto do jira, para que você possa criar o issue.
    Ao criar um issue você deve me avisar qual é o id e a Key do issue, assim como o link para que eu possa abrir-lo no Jira
    Para casos de escrita de de histórias/Storys, você deve utilizar o formato BDD, com cenários e critérios de aceite.
    As storys precisam ter formatação markdown na sua descrição.
     """),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [42]:
OPENAI_API_KEY = os.environ['OPEN_API_KEY']

model = ChatOpenAI(openai_api_key=OPENAI_API_KEY,temperature=0.5)

model_jira_with_tool = model.bind(functions=[format_tool_to_openai_function(criar_issue_Jira)])

In [43]:
agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"])
) | prompt | model_jira_with_tool | OpenAIFunctionsAgentOutputParser()

In [44]:
conversation_table_name = "JiraSessionTable"
message_history = DynamoDBChatMessageHistory(table_name=conversation_table_name, session_id=id)
memory= ConversationBufferMemory(return_messages=True,memory_key="chat_history",chat_memory=message_history)

In [45]:
agent_executor = AgentExecutor(agent=agent_chain, tools=[criar_issue_Jira], verbose=True, memory=memory)

In [46]:
agent_executor.invoke({"input":"olá, como voê pode me ajudar?"})



> Entering new AgentExecutor chain...
Olá! Posso te ajudar a criar documentações funcionais e também a enviar histórias (storys) para o Jira, se necessário. Como posso te auxiliar hoje?

> Finished chain.


{'input': 'olá, como voê pode me ajudar?',
 'chat_history': [HumanMessage(content='olá, como voê pode me ajudar?'),
  AIMessage(content='Olá! Posso te ajudar a criar documentações funcionais e também a enviar histórias (storys) para o Jira, se necessário. Como posso te auxiliar hoje?'),
  HumanMessage(content='crie uma story teste e envie ao jira'),
  AIMessage(content='Claro! Para criar uma story de teste, preciso de algumas informações. Por favor, forneça o nome do projeto do Jira onde deseja criar a story.'),
  HumanMessage(content='O nome do projeto é GENAIT'),
  AIMessage(content='Desculpe, parece que ocorreu um erro ao tentar enviar a story para o Jira. Para que eu possa criar o issue, você precisará configurar suas credenciais da conta do Jira. Uma vez configuradas, posso criar a story de teste para você. Por favor, me avise quando estiver pronto para continuar.')],
 'output': 'Olá! Posso te ajudar a criar documentações funcionais e também a enviar histórias (storys) para o Jira

In [47]:
agent_executor.invoke({"input":"crie uma story teste e envie ao jira"})



> Entering new AgentExecutor chain...


Creating issue "Criar story de teste"



Invoking: `criar_issue_Jira` with `{'project': 'GENAIT', 'title': 'Criar story de teste', 'description': '# Story de Teste\n\n## Cenários:\n\n- Dado que o usuário acessa a página de login\n  - Quando ele insere suas credenciais corretas\n  - Então ele deve ser redirecionado para a página inicial\n\n## Critérios de Aceitação:\n\n- A página de login deve ter um campo para inserir o nome de usuário e senha\n- A autenticação deve ser validada no backend\n- O redirecionamento para a página inicial deve ser feito com sucesso\n', 'issuetype': 'Story'}`
responded: Claro! Vamos criar uma story de teste e enviá-la para o Jira. Por favor, forneça as informações necessárias para a criação da story.

{'id': '10170', 'key': 'GENAIT-154', 'self': 'https://micaelleosouza.atlassian.net/rest/api/2/issue/10170'}
{'id': '10170', 'key': 'GENAIT-154', 'self': 'https://micaelleosouza.atlassian.net/rest/api/2/issue/10170'}A story de teste foi criada com sucesso no Jira! Aqui estão os detalhes do issue:
- ID:

{'input': 'crie uma story teste e envie ao jira',
 'chat_history': [HumanMessage(content='olá, como voê pode me ajudar?'),
  AIMessage(content='Olá! Posso te ajudar a criar documentações funcionais e também a enviar histórias (storys) para o Jira, se necessário. Como posso te auxiliar hoje?'),
  HumanMessage(content='crie uma story teste e envie ao jira'),
  AIMessage(content='Claro! Para criar uma story de teste, preciso de algumas informações. Por favor, forneça o nome do projeto do Jira onde deseja criar a story.'),
  HumanMessage(content='O nome do projeto é GENAIT'),
  AIMessage(content='Desculpe, parece que ocorreu um erro ao tentar enviar a story para o Jira. Para que eu possa criar o issue, você precisará configurar suas credenciais da conta do Jira. Uma vez configuradas, posso criar a story de teste para você. Por favor, me avise quando estiver pronto para continuar.'),
  HumanMessage(content='olá, como voê pode me ajudar?'),
  AIMessage(content='Olá! Posso te ajudar a criar d

In [48]:
agent_executor.invoke({"input":"O nome do projeto é GENAIT"})



> Entering new AgentExecutor chain...


Creating issue "Criar story de teste"



Invoking: `criar_issue_Jira` with `{'project': 'GENAIT', 'title': 'Criar story de teste', 'description': '## Story de Teste\nComo um usuário\nEu quero uma story de teste\nPara verificar a integração com o Jira', 'issuetype': 'Story'}`


{'id': '10171', 'key': 'GENAIT-155', 'self': 'https://micaelleosouza.atlassian.net/rest/api/2/issue/10171'}
{'id': '10171', 'key': 'GENAIT-155', 'self': 'https://micaelleosouza.atlassian.net/rest/api/2/issue/10171'}A story de teste foi criada com sucesso no Jira! Aqui estão os detalhes do issue:
- ID: 10171
- Key: GENAIT-155
- [Link para o issue no Jira](https://micaelleosouza.atlassian.net/browse/GENAIT-155)

Você pode acessar o link acima para visualizar a story no Jira. Se precisar de mais alguma coisa, estou à disposição!

> Finished chain.


{'input': 'O nome do projeto é GENAIT',
 'chat_history': [HumanMessage(content='olá, como voê pode me ajudar?'),
  AIMessage(content='Olá! Posso te ajudar a criar documentações funcionais e também a enviar histórias (storys) para o Jira, se necessário. Como posso te auxiliar hoje?'),
  HumanMessage(content='crie uma story teste e envie ao jira'),
  AIMessage(content='Claro! Para criar uma story de teste, preciso de algumas informações. Por favor, forneça o nome do projeto do Jira onde deseja criar a story.'),
  HumanMessage(content='O nome do projeto é GENAIT'),
  AIMessage(content='Desculpe, parece que ocorreu um erro ao tentar enviar a story para o Jira. Para que eu possa criar o issue, você precisará configurar suas credenciais da conta do Jira. Uma vez configuradas, posso criar a story de teste para você. Por favor, me avise quando estiver pronto para continuar.'),
  HumanMessage(content='olá, como voê pode me ajudar?'),
  AIMessage(content='Olá! Posso te ajudar a criar documentaçõ